In [2]:
import pandas as pd
import numpy as np
from scipy import stats

In [3]:
data_dir = "./data/density.csv"

In [22]:


data = pd.DataFrame.from_csv(data_dir, encoding = "ISO-8859-1")
data.index = pd.Index(map(lambda x: x.year, data.index))
data = data.reset_index()
data = data.drop([data.columns[0], data.columns[2], data.columns[4]], axis=1)
data = data.rename(columns={"index": "Year"})
data.head()

/home/alex/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  This is separate from the ipykernel package so we can avoid doing imports until


,GEO,Value
0,BE100,"7,319.4"
1,BE211,"1,073.7"
2,BE212,672.1
3,BE213,335.0
4,BE221,471.8


In [23]:
data["Value"] = data.Value.apply(lambda x: float(np.nan if x == ":" else x.replace(",", "")))

data

,GEO,Value
0,BE100,7319.4
1,BE211,1073.7
2,BE212,672.1
3,BE213,335.0
4,BE221,471.8
5,BE222,273.6
6,BE223,320.8
7,BE231,603.8
8,BE232,589.9
9,BE233,251.7


In [13]:
nuts3 = all_data[nuts_val == 3]
nuts3.to_csv("./data/nuts3.csv")
nuts3.head()

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
GEO,,,,,,,,,,,,,,,,,,,
AL011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,140956.0,137722.0,134332.0,131054.0,129056.0,125579.0,120978.0
AL012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,271947.0,274029.0,276058.0,277989.0,280205.0,284823.0,289626.0
AL013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,87762.0,86082.0,84306.0,82428.0,81294.0,79559.0,77395.0
AL014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,137782.0,135703.0,133748.0,131829.0,130258.0,129019.0,126800.0
AL015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,222611.0,219548.0,216477.0,213148.0,210168.0,207924.0,204994.0


In [14]:
len(nuts3) - nuts3.count(), nuts3.values.shape

(2000    349
 2001    260
 2002    263
 2003    192
 2004    190
 2005    171
 2006    171
 2007    160
 2008    103
 2009    103
 2010     89
 2011     89
 2012     69
 2013     57
 2014     25
 2015     25
 2016     25
 2017      0
 2018    137
 dtype: int64, (982, 19))

In [222]:
ind = nuts0.index.values
X = nuts0.values
y, X = X[:, -1], X[:, :-1]
y.shape, X.shape

((37,), (37, 18))

In [223]:
def get_linreg_preds(X, start_year, end_year):
    res = []
    xs = np.arange(start_year, end_year)
    for ys in X:
        slope, intercept, r_value, p_value, std_err = stats.linregress(xs,ys)
        squared_error = np.square(y[0] - (intercept + slope * end_year))
        res.append(round(intercept + slope * end_year))
    return np.array(res)

In [224]:
linreg_preds = get_linreg_preds(X, 2000, 2018)

In [225]:
n3data = nuts3.loc[:, 2005:2017].dropna().values

In [235]:
linreg_preds_nuts3 = get_linreg_preds(n3data[:, :-1], 2005, 2017)
np.mean(np.square((linreg_preds_nuts3 - n3data[:, -1])/n3data[:, -1]))

0.00028161393898762924

In [226]:
n3data, n3data.shape

(array([[ 37522.,  37413.,  37481., ...,  37622.,  37692.,  37752.],
        [143094., 144205., 145118., ..., 153637., 155809., 156735.],
        [ 97416.,  97509.,  97463., ...,  97097.,  97510.,  97455.],
        ...,
        [661212., 660951., 660765., ..., 655359., 653024., 651509.],
        [797899., 800228., 802659., ..., 819977., 820697., 822310.],
        [774356., 777090., 779663., ..., 795565., 796650., 798103.]]),
 (811, 13))

In [227]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
n3data_norm = (n3data - np.mean(n3data))/np.var(n3data)#normalize(n3data, axis=1, norm='l1')
X, Y = n3data_norm[:, :-1], n3data_norm[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [268]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.hidden = torch.nn.Linear(n_feature, n_hidden)   # hidden layer
        self.predict = torch.nn.Linear(n_hidden, n_output)   # output layer

    def forward(self, x):
        x = F.relu(self.hidden(x))      # activation function for hidden layer
        x = self.predict(x)             # linear output
        return x


net = Net(1, 1, 1)
print(net)


Net(
  (hidden): Linear(in_features=1, out_features=1, bias=True)
  (predict): Linear(in_features=1, out_features=1, bias=True)
)


In [273]:
device = torch.device('cpu')
x = torch.from_numpy(X_train).float()
y = torch.from_numpy(y_train).float()
print(x.size())
print(y.size())
model = Net(12, 6, 1)
loss_fn = torch.nn.MSELoss(reduction='mean')
model

torch.Size([608, 12])
torch.Size([608])


Net(
  (hidden): Linear(in_features=12, out_features=6, bias=True)
  (predict): Linear(in_features=6, out_features=1, bias=True)
)

In [274]:
learning_rate = 1e-2
for t in range(1000):
    y_pred = model(x)

    loss = loss_fn(y_pred, y)
    if t % 100 == 0:
        print(t, loss.item())

    model.zero_grad()

    loss.backward()

    with torch.no_grad():
        for param in model.parameters():
            param.data -= learning_rate * param.grad


0 0.09254413098096848
100 0.0010004823561757803
200 9.824393600865733e-06
300 9.539967038563191e-08
400 9.296720060447683e-10
500 1.3417110304725366e-11
600 4.86369755234306e-12
700 4.745244995552067e-12
800 4.745244995552067e-12
900 4.745244995552067e-12


In [275]:
def score(x, y):
    x = torch.from_numpy(x).float()
    x = model(x).detach().numpy().reshape(x.size()[0])
    y = y * np.var(n3data) + np.mean(n3data)
    y_pred = x * np.var(n3data) + np.mean(n3data)
    return np.mean(np.square((y_pred - y)/y)), y_pred
score(X_train, y_train)[0]

9.00788940731023

In [276]:
score(X_test, y_test)[0]

3.7858463639531736

In [243]:
for i in range(10):
    print(score(X_test, y_test)[1][i], y_test[i] * np.var(n3data) + np.mean(n3data))

335464.88 191685.0
352113.8 466947.0
328805.3 53556.0
372092.56 761763.0
332135.1 122698.0
332135.1 200716.0
342124.47 289584.0
405390.44 1348462.0
352113.8 426140.0
345454.25 321377.0
